In [ ]:
# ============================================================================
# Cell 1: นำเข้าไลบรารี่และตั้งค่าเส้นทาง (Imports & Configuration)
# ============================================================================

# ========== ส่วน A: นำเข้าไลบรารี่ที่จำเป็น ==========
import pandas as pd          # 📊 ประมวลผลข้อมูล (DataFrames, การอ่าน/บันทึก CSV)
import numpy as np           # 🔢 คำนวณตัวเลข (arrays, mathematical functions)
from pathlib import Path     # 📁 จัดการเส้นทางไฟล์แบบ cross-platform

# ========== ส่วน B: กำหนด Path ไฟล์ ==========
PROJECT_ROOT = Path("..").resolve()  # เช่น /Users/thanaporn/Desktop/EURO_H1_AI
DATA_DIR = PROJECT_ROOT / "data_csv"  # โฟลเดอร์ข้อมูล CSV
RAW_D1_PATH = DATA_DIR / "EURUSD_D1.csv"  # ไฟล์ OHLC daily (D1 timeframe)

# ========== ส่วน C: ค่าคงที่สำหรับแปลงหน่วย ==========
# EURUSD ใช้ 4 decimal places → 1 pip = 0.0001
# ถ้าค่ารากา 0.0001 → คำนวณเป็น 1 pip
# ตัวคูณ 10000 ใช้เพื่อแปลงจาก decimal เป็น pips count
# ตัวอย่าง: (1.5200 - 1.5100) * 10000 = 100 pips
PIPS_MULTIPLIER = 10000

In [154]:
# ============================================================================
# Cell 2: ฟังก์ชันโหลดไฟล์ OHLC (Open-High-Low-Close) - Daily Data
# ============================================================================
# จุดประสงค์: อ่านไฟล์ CSV ที่มี OHLC data → validate + format ข้อมูล → return DataFrame

def load_ohlc_d1(path: Path, sep: str = "\t") -> pd.DataFrame:
    """
    โหลดไฟล์ OHLC (Daily D1) จากไฟล์ CSV
    
    ขั้นตอน:
    1. ตรวจสอบว่าไฟล์มีอยู่หรือไม่
    2. อ่านไฟล์ CSV เข้า DataFrame
    3. ตรวจสอบว่ามีคอลัมน์ที่จำเป็น (Time, Open, High, Low, Close)
    4. แปลง Time เป็น datetime format
    5. แปลง OHLC + Volume/Spread เป็น numeric (float/int)
    6. เรียงข้อมูลตามลำดับเวลา (chronological order)
    
    Parameters:
    -----------
    path : Path - เส้นทางไฟล์ (เช่น /path/to/EURUSD_D1.csv)
    sep : str - ตัวคั่นคอลัมน์ ("," สำหรับ CSV, "\\t" สำหรับ TSV)
    
    Returns:
    --------
    pd.DataFrame - ข้อมูล OHLC ที่ cleaned พร้อมใช้งาน
    """
    # ========== 1. ตรวจสอบว่าไฟล์มีอยู่ ==========
    if not path.exists():
        raise FileNotFoundError(f"File not found: {path}")
    
    # ========== 2. อ่านไฟล์ CSV ==========
    df = pd.read_csv(path, sep=sep)
    
    # ========== 3. ตรวจสอบคอลัมน์ที่จำเป็น ==========
    # ต้องมี Time, Open, High, Low, Close อย่างน้อย
    required = {"Time", "Open", "High", "Low", "Close"}
    if not required.issubset(set(df.columns)):
        raise ValueError(f"File must contain columns: {required}, got {df.columns.tolist()}")
    
    # ========== 4 & 5. แปลง Data Types ==========
    # Time → datetime (ชนิดข้อมูลสำหรับเวลา)
    df["Time"] = pd.to_datetime(df["Time"])
    
    # OHLC → float (ทศนิยม) โดยแปลงค่าที่ไม่ถูกต้องเป็น NaN
    float_cols = ["Open", "High", "Low", "Close"]
    for c in float_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    
    # Volume/Spread → numeric (ถ้ามีอยู่ในไฟล์)
    for c in ["Volume", "Spread"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    
    # ========== 6. เรียงตามลำดับเวลา (ต้องเป็น chronological order) ==========
    # sort_values('Time') = เรียงจากเก่าไปใหม่
    # reset_index(drop=True) = สร้าง index ใหม่ (0, 1, 2, ...)
    df = df.sort_values("Time").reset_index(drop=True)
    return df


# ========== 7. เรียกใช้ฟังก์ชัน ==========
# โหลดข้อมูล D1 (Daily) EURUSD จากไฟล์ CSV
# sep="\\t" เพราะไฟล์มีข้อมูลคั่นด้วย tab
raw_df = load_ohlc_d1(RAW_D1_PATH, sep="\t")

# ========== 8. ดูตัวอย่างข้อมูล ==========
# แสดง 5 แถวแรก เพื่อตรวจสอบว่าโหลดสำเร็จ
raw_df.head()

,Time,Open,High,Low,Close,Volume,Spread
0,2009-11-26,1.51073,1.51381,1.49404,1.49682,70915,9
1,2009-11-27,1.49653,1.49985,1.48264,1.49853,76695,11
2,2009-11-30,1.49931,1.50825,1.49702,1.50343,79595,9
3,2009-12-01,1.50351,1.51194,1.49719,1.50885,79059,9
4,2009-12-02,1.50881,1.51096,1.50290,1.50635,80300,9


In [ ]:
# ============================================================================
# Cell 3: คำนวณ ATR (Average True Range) และฟีเจอร์รากาแท่งเทียน
# ============================================================================
# จุดประสงค์: สร้างฟีเจอร์ (price features) ที่อธิบายการเคลื่อนไหว/ความผันผวนรายวัน

# ========== ส่วน A: ATR (Volatility Indicator) ==========
def calculate_atr(df: pd.DataFrame, period: int = 14) -> pd.Series:
    """
    คำนวณ Average True Range (ATR) - วัดความผันผวน/ขนาดการเคลื่อนไหวเฉลี่ย
    
    ATR = ค่าเฉลี่ยเคลื่อนที่ (moving average) ของ True Range เป็นระยะเวลา 14 วัน
    True Range = max(High-Low, |High-Close(t-1)|, |Low-Close(t-1)|)
    
    หมายถึง: ความผันผวนที่คาดหวังได้ตามปกติศาสตร์
    """
    high = df["High"]
    low = df["Low"]
    close = df["Close"]
    
    # คำนวณ 3 องค์ประกอบของ True Range:
    tr1 = high - low                        # ช่วงรากาวันนี้
    tr2 = (high - close.shift(1)).abs()   # ห่างระหว่าง High วันนี้ กับ Close เมื่อวาน
    tr3 = (low - close.shift(1)).abs()    # ห่างระหว่าง Low วันนี้ กับ Close เมื่อวาน
    
    # True Range = ค่ามากที่สุด ของ 3 องค์ประกอบ
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    
    # ATR = ค่าเฉลี่ยเคลื่อนที่ที่ 14 วัน (moving average) ของ True Range
    atr = tr.rolling(window=period, min_periods=period).mean()
    return atr


# ========== ส่วน B: ฟีเจอร์รากาแท่งเทียน (Candlestick Features) ==========
def add_daily_price_features(df: pd.DataFrame, pips_multiplier: int = 10000) -> pd.DataFrame:
    """
    สร้างฟีเจอร์ที่อธิบายพฤติกรรมรากาแท่งเทียน เพื่อใช้สำหรับโมเดล
    
    ฟีเจอร์ที่เพิ่ม:
    - daily_change_pips: ราคาเปลี่ยนแปลงแต่ละ pips (vs Close เมื่อวาน)
    - daily_change_pct: ราคาเปลี่ยนแปลงแต่ละเปอร์เซนต์
    - body_size_pips: ขนาด \"ลำตัว\" แท่งเทียน (|Close - Open|)
    - upper_shadow_pips: เงาบน (ส่วนสูงสุดของแท่ง)
    - lower_shadow_pips: เงาล่าง (ส่วนตำสุดของแท่ง)
    - total_range_pips: ช่วงราคาทั้งหมด (High - Low)
    - atr14: ความผันผวน 14 วัน
    """
    df = df.copy()  # สำเนา DataFrame เพื่อไม่ให้เปลี่ยนแปลงข้อมูลต้นฉบับ
    
    # ========== 1) การเปลี่ยนแปลงรากา (Price Change) ==========
    # เปลี่ยนแปลงจาก Close เมื่อวาน vs Close วันนี้
    # shift(1) = ข้อมูลแถวก่อนหน้า (เมื่อวาน)
    df["daily_change_pips"] = (df["Close"] - df["Close"].shift(1)) * pips_multiplier
    df["daily_change_pct"] = (df["Close"] - df["Close"].shift(1)) / df["Close"].shift(1) * 100
    
    # ========== 2) โครงสร้างสร้างแท่งเทียน (Candlestick Structure) ==========
    # \"Body\" = ลำตัวแท่ง = |Close - Open|
    df["body_size_pips"] = (df["Close"] - df["Open"]).abs() * pips_multiplier
    
    # \"Upper Shadow\" = เงาบน = ส่วนที่สูงกว่า max(Open, Close)
    df["upper_shadow_pips"] = (df["High"] - df[["Open", "Close"]].max(axis=1)) * pips_multiplier
    
    # \"Lower Shadow\" = เงาล่าง = ส่วนที่ตำกว่า min(Open, Close)
    df["lower_shadow_pips"] = (df[["Open", "Close"]].min(axis=1) - df["Low"]) * pips_multiplier
    
    # \"Total Range\" = ช่วงราคาวันนี้ = High - Low
    df["total_range_pips"] = (df["High"] - df["Low"]) * pips_multiplier
    
    # ========== 3) ATR (Volatility) ==========
    df["atr14"] = calculate_atr(df, period=14) * pips_multiplier
    
    return df


# ========== 4. เรียกใช้ฟังก์ชัน ==========
# เพิ่มฟีเจอร์ทั้งหมดให้กับ raw_df
d1_feat = add_daily_price_features(raw_df, pips_multiplier=PIPS_MULTIPLIER)

# ========== 5. ดูผลลัพธ์ ==========
# แสดง 5 แถวสุดท้าย (ข้อมูลล่าสุด) เพื่อตรวจสอบฟีเจอร์ใหม่
d1_feat.tail()

,Time,Open,High,Low,Close,Volume,Spread,daily_change_pips,daily_change_pct,body_size_pips,upper_shadow_pips,lower_shadow_pips,total_range_pips,atr14
4176,2025-12-02,1.16069,1.16290,1.15910,1.16280,74054,4,21.4,0.184378,21.1,1.0,15.9,38.0,53.864286
4177,2025-12-03,1.16280,1.16775,1.16277,1.16661,95663,4,38.1,0.327657,38.1,11.4,0.3,49.8,51.921429
4178,2025-12-04,1.16662,1.16819,1.16409,1.16431,79501,3,-23.0,-0.197152,23.1,15.7,2.2,41.0,51.450000
4179,2025-12-05,1.16433,1.16717,1.16279,1.16388,75183,3,-4.3,-0.036932,4.5,28.4,10.9,43.8,52.050000
4180,2025-12-08,1.16406,1.16718,1.16394,1.16579,16567,3,19.1,0.164106,17.3,13.9,1.2,32.4,51.814286


In [ ]:
# ============================================================================
# Cell 4: จำแนก Daily Bias (ทิศทาง) และ Daily Strength (ความแรง)
# ============================================================================
# จุดประสงค์: 
# จำแนกแต่ละวันว่าเป็น bullish/bearish/neutral และแรง/ปกติ/อ่อนแอ
# ใช้เพื่อให้โมเดล learning ได้วัน-ไหนมี-แนวโน้มขึ้น/ลง แล้วแรงแค่ไหน

def classify_daily_bias_and_strength(
    df: pd.DataFrame,
    neutral_threshold_pct: float = 0.15,           # ขีดจำกัด % สำหรับ neutral
    strong_threshold_multiplier: float = 1.5,      # ตัวคูณ ATR สำหรับ \"strong\"
) -> pd.DataFrame:
    """
    คำแนก 2 มิติ:
    
    มิติที่ 1: BIAS (ทิศทาง) - บอกว่ารากาขึ้นหรือลง
      - bullish (1): รากาขึ้น มากกว่า 0.15%
      - bearish (-1): รากาลง มากกว่า 0.15%
      - neutral (0): รากาเปลี่ยนแปลงเล็กน้อย (-0.15% to +0.15%)
    
    มิติที่ 2: STRENGTH (ความแรง) - บอกว่าการเคลื่อนไหวแรงแค่ไหน
      - strong (2): ขนาดการเปลี่ยนแปลง > 1.5 × ATR14 (ผันผวนอย่างนอก)
      - normal (1): ขนาดการเปลี่ยนแปลง > 0.5 × ATR14
      - weak (0): ขนาดการเปลี่ยนแปลง <= 0.5 × ATR14 (ดึม/ไม่ปะตัว)
    """
    df = df.copy()  # สำเนา DataFrame
    
    # ========== ส่วน A: จำแนก BIAS (ทิศทาง) ==========
    # สร้างเงื่อนไข 2 เงื่อนไข:
    # - Bullish: daily_change_pct > 0.15%
    # - Bearish: daily_change_pct < -0.15%
    cond_bias = [
        df["daily_change_pct"] > neutral_threshold_pct,    # ขึ้น (bullish)
        df["daily_change_pct"] < -neutral_threshold_pct,   # ลง (bearish)
    ]
    choice_bias = ["bullish", "bearish"]
    
    # np.select: ตรวจเงื่อนไข + return ค่าที่เลือก (ค่า default = \"neutral\")
    df["daily_bias"] = np.select(cond_bias, choice_bias, default="neutral")
    
    # Encode: แปลง text เป็นตัวเลข (สำหรับโมเดล)
    # bullish=1, neutral=0, bearish=-1
    bias_map = {"bullish": 1, "neutral": 0, "bearish": -1}
    df["daily_bias_encoded"] = df["daily_bias"].map(bias_map)
    
    # ========== ส่วน B: จำแนก STRENGTH (ความแรง) ==========
    # Strength = มองขนาดการเปลี่ยนแปลง (pips) เทียบกับ ATR14
    abs_change_pips = df["daily_change_pips"].abs()  # ขนาดการเปลี่ยนแปลง (ไม่สนใจเครื่องหมาย)
    strong_threshold = df["atr14"] * strong_threshold_multiplier  # ขีด \"strong\" = 1.5 × ATR
    
    # เงื่อนไข:
    # - strong: abs_change_pips > (1.5 × ATR14)
    # - normal: abs_change_pips > (0.5 × ATR14)
    # - weak (default): อื่นๆ
    cond_strength = [
        abs_change_pips > strong_threshold,        # แรงมากกว่า 1.5×ATR
        abs_change_pips > df["atr14"] * 0.5,       # ปกติระหว่าง 0.5-1.5×ATR
    ]
    choice_strength = ["strong", "normal"]
    df["daily_strength"] = np.select(cond_strength, choice_strength, default="weak")
    
    # Encode: strong=2, normal=1, weak=0
    strength_map = {"strong": 2, "normal": 1, "weak": 0}
    df["daily_strength_encoded"] = df["daily_strength"].map(strength_map)
    
    return df


# ========== 1. เรียกใช้ฟังก์ชัน ==========
d1_feat = classify_daily_bias_and_strength(d1_feat)

# ========== 2. ดูผลลัพธ์ ==========
# แสดง: Time, Close, daily_change_pips (เท่าไหร่เปลี่ยนแปลง), bias, strength
# ช่วยให้เห็นการจำแนกว่าถูกต้องไหม
d1_feat[["Time", "Close", "daily_change_pips", "daily_bias", "daily_strength"]].tail()

,Time,Close,daily_change_pips,daily_bias,daily_strength
4176,2025-12-02,1.16280,21.4,bullish,weak
4177,2025-12-03,1.16661,38.1,bullish,normal
4178,2025-12-04,1.16431,-23.0,bearish,weak
4179,2025-12-05,1.16388,-4.3,neutral,weak
4180,2025-12-08,1.16579,19.1,bullish,weak


In [ ]:
# ============================================================================\n# Cell 5: สร้าง Lagged Features (ข้อมูลเมื่อวาน) + Regime (สถานะความผันผวน/โครงสร้างสร้างแท่ง)\n# ============================================================================\n# จุดประสงค์: \n# 1. เพิ่ม \"ข้อมูลเมื่อวาน\" เป็น feature ให้โมเดล (temporal context)\n# 2. จำแนก \"regime\" = สถานะความผันผวน + โครงสร้างสร้างแท่งของวันนี้\n# 3. สร้าง \"target\" = regime ของวันถัดไป (ที่เราต้องการทำนาย)\n\ndef add_prev_day_and_regime_targets(df: pd.DataFrame) -> pd.DataFrame:\n    \"\"\"\n    สร้าง 3 ประเภทข้อมูล:\n    \n    (A) LAGGED FEATURES: ข้อมูลของวันก่อนหน้า (shift by 1 day)\n        - prev_daily_change_pips, prev_daily_change_pct\n        - prev_body_size_pips, prev_total_range_pips, prev_atr14\n        → ใช้เพื่อให้โมเดล \"จำ\" วันเมื่อวาน และสามารถจำแนกแนวโน้มได้\n    \n    (B) REGIME (ปัจจุบัน): สถานะของวันนี้\n        - vol_regime: high_vol (เพลิดปรี่) หรือ low_vol (เงียบ)\n        - struct_regime: trending (แนวโน้ม) หรือ choppy (ผันผวน/ไม่ชัด)\n    \n    (C) TARGET (อนาคต): regime ของวันถัดไป (ที่ต้องการทำนาย)\n        - target_vol_regime_next, target_struct_regime_next\n    \"\"\"\n    df = df.copy()  # สำเนา\n    \n    # ========== ส่วน A: Lagged Features (ข้อมูล t-1) ==========\n    # shift(1) = ข้อมูลแถวก่อนหน้า (เมื่อวาน)\n    # ทำให้โมเดล \"เห็น\" วันเมื่อวาน และสามารถจำแนกแนวโน้มได้\n    print(\"🔄 Creating lagged features (yesterday's data)...\")\n    df[\"prev_daily_change_pips\"] = df[\"daily_change_pips\"].shift(1)    # เมื่อวาน-เปลี่ยนแปลง pips เท่าไหร่\n    df[\"prev_daily_change_pct\"] = df[\"daily_change_pct\"].shift(1)      # เมื่อวาน-เปลี่ยนแปลง % เท่าไหร่\n    df[\"prev_body_size_pips\"] = df[\"body_size_pips\"].shift(1)          # เมื่อวาน-body ใหญ่เท่าไหร่\n    df[\"prev_total_range_pips\"] = df[\"total_range_pips\"].shift(1)      # เมื่อวาน-range เท่าไหร่\n    df[\"prev_atr14\"] = df[\"atr14\"].shift(1)                            # เมื่อวาน-ATR เท่าไหร่\n    \n    # ========== ส่วน B: ปัจจุบัน REGIME (สถานะวันนี้) ==========\n    print(\"📊 Computing regime classifications...\")\n    \n    # --- (B1) Vol_regime: ความผันผวน ---\n    # หลัก: เปรียบเทียบขนาดการเปลี่ยนแปลง (pips) กับ ATR\n    # ถ้า |price_change| > threshold × ATR → \"explosive\" = high_vol\n    # ถ้า |price_change| <= threshold × ATR → \"quiet\" = low_vol\n    eps = 1e-8  # ค่ากันหารด้วยศูนย์\n    vol_ratio = (df[\"daily_change_pips\"].abs() / (df[\"atr14\"].abs() + eps))\n    \n    # 🔥 threshold = 0.6 (ปรับจาก 1.0 → 0.6 เพื่อให้ high_vol มากขึ้น)\n    # ความหมาย: ถ้า price_change > 0.6 × ATR → high_vol\n    df[\"vol_regime\"] = np.where(vol_ratio > 0.6, \"high_vol\", \"low_vol\")\n    vol_map = {\"low_vol\": 0, \"high_vol\": 1}\n    df[\"vol_regime_encoded\"] = df[\"vol_regime\"].map(vol_map)  # 0 or 1\n    \n    # --- (B2) Struct_regime: โครงสร้างสร้างแท่ง ---\n    # หลัก: ดูสัดส่วน body / range\n    # body_ratio สูง (>= 0.55) → ปิดได้ดี \"trending\" (ทิศชัด)\n    # body_ratio ต่ำ (< 0.55) → ปิดไม่ได้ดี \"choppy\" (ผันผวน/ไม่ชัด)\n    body_ratio = df[\"body_size_pips\"] / (df[\"total_range_pips\"].abs() + eps)\n    df[\"body_ratio\"] = body_ratio.clip(0.0, 1.0)  # Clip: ไม่ให้เกิน 0-1 range\n    \n    df[\"struct_regime\"] = np.where(df[\"body_ratio\"] >= 0.55, \"trending\", \"choppy\")\n    struct_map = {\"choppy\": 0, \"trending\": 1}\n    df[\"struct_regime_encoded\"] = df[\"struct_regime\"].map(struct_map)  # 0 or 1\n    \n    # ข้อมูล regime ของเมื่อวาน (lagged)\n    df[\"prev_vol_regime_encoded\"] = df[\"vol_regime_encoded\"].shift(1)\n    df[\"prev_struct_regime_encoded\"] = df[\"struct_regime_encoded\"].shift(1)\n    \n    # ========== ส่วน C: TARGET (เป้าหมายทำนาย - วันถัดไป) ==========\n    # shift(-1) = ข้อมูลแถวถัดไป (พรุ่งนี้ ← future)\n    # นี่คือค่าที่เราต้องการให้โมเดล PREDICT\n    print(\"🎯 Creating target labels (next day's regime)...\")\n    df[\"target_vol_regime_next\"] = df[\"vol_regime_encoded\"].shift(-1)          # vol regime ของพรุ่งนี้\n    df[\"target_struct_regime_next\"] = df[\"struct_regime_encoded\"].shift(-1)    # struct regime ของพรุ่งนี้\n    \n    # ========== ส่วน D: ทำความสะอาด ==========\n    # dropna(): ลบแถวที่มี NaN (เกิดจาก shift ต้นและท้าย)\n    # reset_index(drop=True): สร้าง index ใหม่ (0, 1, 2, ...)\n    print(\"🧹 Dropping NaN rows from lagged/target columns...\")\n    df = df.dropna().reset_index(drop=True)\n    \n    return df\n\n\n# ========== 1. เรียกใช้ ==========\ndataset = add_prev_day_and_regime_targets(d1_feat)\n\n# ========== 2. ดูผลลัพธ์ ==========\n# แสดง first 5 rows และ last 5 rows เพื่อตรวจสอบ\ndataset.head(), dataset.tail()

(        Time     Open     High      Low    Close  Volume  Spread  \
 0 2009-12-16  1.45398  1.45903  1.45023  1.45219   79373       9   
 1 2009-12-17  1.45204  1.45306  1.43035  1.43427   80893       9   
 2 2009-12-18  1.43429  1.44105  1.42582  1.43364   77356       9   
 3 2009-12-21  1.43301  1.43727  1.42636  1.42834   80115       9   
 4 2009-12-22  1.42838  1.43320  1.42149  1.42518   79862       9   
 
    daily_change_pips  daily_change_pct  body_size_pips  ...  prev_atr14  \
 0              -16.4         -0.112805            17.9  ...  146.357143   
 1             -179.2         -1.233998           177.7  ...  138.521429   
 2               -6.3         -0.043925             6.5  ...  142.450000   
 3              -53.0         -0.369688            46.7  ...  145.307143   
 4              -31.6         -0.221236            32.0  ...  142.564286   
 
    vol_regime  vol_regime_encoded  body_ratio struct_regime  \
 0     low_vol                   0    0.203409        choppy  

In [ ]:
# ============================================================================\n# Cell 6: เลือก (Filter) คอลัมน์ที่ต้องการ + บันทึก Dataset เป็น CSV\n# ============================================================================\n# จุดประสงค์: \n# 1. เลือกเฉพาะคอลัมน์ที่ต้องการ (ลบ temp columns ที่ไม่ใช้)\n# 2. บันทึก Dataset ลงไฟล์ CSV เพื่อใช้ต่อไป (reproducibility)\n\n# ========== 1. กำหนดคอลัมน์ที่เก็บ ==========\n# cols_keep = รายชื่อ 26 คอลัมน์ที่ต้องการ สำหรับโมเดล\n# ประกอบด้วย:\n# - Time: timestamp\n# - OHLC: ราคาหลัก\n# - Daily features: daily_change, body_size, shadows, range, atr\n# - Lagged features (วันก่อน): prev_change, prev_range, prev_atr\n# - Regime (วันนี้): vol_regime, struct_regime, encoded versions\n# - Regime lagged (วันเมื่อวาน): prev_vol_regime_encoded, prev_struct_regime_encoded\n# - Target (พรุ่งนี้): target_vol_regime_next, target_struct_regime_next ← ให้โมเดลทำนาย\nprint(\"📋 Selecting important columns for model...\")\ncols_keep = [\n    # เวลา\n    \"Time\",\n    \n    # OHLC ราคาหลัก\n    \"Open\", \"High\", \"Low\", \"Close\",\n    \n    # Daily price features (วันนี้)\n    \"daily_change_pips\", \"daily_change_pct\",\n    \"body_size_pips\", \"upper_shadow_pips\", \"lower_shadow_pips\", \"total_range_pips\",\n    \"atr14\",\n    \"body_ratio\",\n    \n    # Lagged features (เมื่อวาน) - บอกโมเดล \"เมื่อวาน-ไรไหร่\"\n    \"prev_daily_change_pips\", \"prev_daily_change_pct\", \"prev_body_size_pips\",\n    \"prev_total_range_pips\", \"prev_atr14\",\n    \n    # Regime ปัจจุบัน (วันนี้) - ข้อมูล explanatory\n    \"vol_regime\", \"vol_regime_encoded\",\n    \"struct_regime\", \"struct_regime_encoded\",\n    \n    # Regime lagged (เมื่อวาน) - temporal context\n    \"prev_vol_regime_encoded\", \"prev_struct_regime_encoded\",\n    \n    # Target (พรุ่งนี้) ← \"ลัก ษณ์เฉพาะ\" ที่โมเดลต้องทำนาย!\n    \"target_vol_regime_next\", \"target_struct_regime_next\",\n]\n\n# ========== 2. สร้าง DataFrame ใหม่ที่มีเฉพาะ cols_keep ==========\n# copy() = สำเนา ไม่เปลี่ยนแปลง dataset ต้นฉบับ\ndataset_final = dataset[cols_keep].copy()\n\n# ========== 3. ดูตัวอย่าง ==========\nprint(f\"✅ Selected {len(cols_keep)} columns\")\ndataset_final.tail()  # แสดง 5 แถวสุดท้าย\n\n# ========== 4. สร้างโฟลเดอร์ OUTPUT (ถ้ายังไม่มี) ==========\nprint(\"\\n💾 Saving dataset...\")\nOUTPUT_DIR = PROJECT_ROOT / \"datasets\"\nOUTPUT_DIR.mkdir(exist_ok=True)  # exist_ok=True: ไม่ error ถ้าโฟลเดอร์มีอยู่แล้ว\nOUT_PATH = OUTPUT_DIR / \"model1_regime_dataset.csv\"\n\n# ========== 5. บันทึก CSV ==========\n# index=False: ไม่บันทึก row index (0,1,2,..)\ndataset_final.to_csv(OUT_PATH, index=False)\n\nprint(f\"✅ Saved regime dataset for Model 1 to: {OUT_PATH}\")\nprint(f\"📊 Rows: {len(dataset_final)}, Columns: {len(dataset_final.columns)}\")\nprint(f\"🎯 Dataset Ready for Model Training!\")

✅ Saved regime dataset for Model 1 to: /Users/thanaporn/Desktop/EURO_H1_AI/datasets/model1_regime_dataset.csv
⚡ Rows: 4166, Columns: 26


#################################################################################################################################################

In [ ]:
# Cell 7 (UPDATED): โหลด dataset สำหรับ Model 1 (Regime)

import pandas as pd
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
DATASET_PATH = PROJECT_ROOT / "datasets" / "model1_regime_dataset.csv"

df = pd.read_csv(DATASET_PATH)
df["Time"] = pd.to_datetime(df["Time"])

print(df.shape)
df.head()


In [ ]:
# Cell 8 (same logic, ใช้กับ df ปัจจุบัน)

df = df.sort_values("Time").reset_index(drop=True)

print("Columns:")
for c in df.columns:
    print("-", c)

df.tail(3)


In [ ]:
# Cell 9 (UPDATED): เลือก features + target สำหรับ Volatility Regime

TARGET_COL = "target_vol_regime_next"  # 0 = low_vol, 1 = high_vol

feature_cols = [
    # ราคา + พฤติกรรมวันนี้
    "Open", "High", "Low", "Close",
    "daily_change_pips",
    "daily_change_pct",
    "body_size_pips",
    "upper_shadow_pips",
    "lower_shadow_pips",
    "total_range_pips",
    "atr14",
    "body_ratio",
    
    # ข้อมูลเมื่อวาน
    "prev_daily_change_pips",
    "prev_daily_change_pct",
    "prev_body_size_pips",
    "prev_total_range_pips",
    "prev_atr14",
    "prev_vol_regime_encoded",
    "prev_struct_regime_encoded",
]

missing = [c for c in feature_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing feature columns: {missing}")

X = df[feature_cols].copy()
y = df[TARGET_COL].copy()

X.shape, y.shape, y.value_counts()


In [ ]:
# Cell 10: แบ่ง train / valid / test ตามลำดับเวลา

n = len(df)
train_end = int(n * 0.7)
valid_end = int(n * 0.85)

X_train = X.iloc[:train_end]
y_train = y.iloc[:train_end]

X_valid = X.iloc[train_end:valid_end]
y_valid = y.iloc[train_end:valid_end]

X_test = X.iloc[valid_end:]
y_test = y.iloc[valid_end:]

print("Train :", X_train.shape, y_train.shape)
print("Valid :", X_valid.shape, y_valid.shape)
print("Test  :", X_test.shape, y_test.shape)

print("\nTarget distribution:")
print("Train:\n", y_train.value_counts())
print("Valid:\n", y_valid.value_counts())
print("Test:\n", y_test.value_counts())


In [ ]:
# Cell 11 (NEW): เทรน RandomForest สำหรับ Volatility Regime แบบไม่ถ่วงฝั่งไหนเป็นพิเศษ

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

rf_regime = RandomForestClassifier(
    n_estimators=400,
    max_depth=9,
    min_samples_leaf=4,
    random_state=42,
    n_jobs=-1,
    class_weight=None,  # ❌ ไม่ถ่วงน้ำหนัก high_vol แล้ว
)

rf_regime.fit(X_train, y_train)

# ใช้ prob สำหรับลอง threshold หลายๆ ค่า
y_valid_proba = rf_regime.predict_proba(X_valid)[:, 1]
y_test_proba = rf_regime.predict_proba(X_test)[:, 1]


In [ ]:
# Cell 12 (NEW): ลองหลาย threshold แล้วหาอันที่ accuracy สูงสุด (เน้นทายถูกรวมเยอะสุด)

from sklearn.metrics import precision_score, recall_score, f1_score

thresholds = [0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6]

best_th = None
best_acc = -1
records = []

print("🔍 THRESHOLD SCAN (VALID) - optimize for OVERALL ACCURACY")
print("th\tacc\tprec0\trec0\tprec1\trec1")

for th in thresholds:
    y_valid_pred = (y_valid_proba >= th).astype(int)
    
    acc = accuracy_score(y_valid, y_valid_pred)
    prec0 = precision_score(y_valid, y_valid_pred, pos_label=0, zero_division=0)
    rec0  = recall_score(y_valid, y_valid_pred, pos_label=0, zero_division=0)
    prec1 = precision_score(y_valid, y_valid_pred, pos_label=1, zero_division=0)
    rec1  = recall_score(y_valid, y_valid_pred, pos_label=1, zero_division=0)
    
    records.append((th, acc))
    
    if acc > best_acc:
        best_acc = acc
        best_th = th
    
    print(f"{th:.2f}\t{acc:.3f}\t{prec0:.3f}\t{rec0:.3f}\t{prec1:.3f}\t{rec1:.3f}")

print("\n✅ BEST THRESHOLD on VALID (by accuracy):", best_th, "acc =", best_acc)

# ใช้ threshold ที่ดีที่สุดกับ valid/test แล้วสรุปเต็มๆ
y_valid_pred_best = (y_valid_proba >= best_th).astype(int)
y_test_pred_best  = (y_test_proba >= best_th).astype(int)

print("\n===== VALID @ best_th =====")
print("Accuracy:", accuracy_score(y_valid, y_valid_pred_best))
print(classification_report(
    y_valid,
    y_valid_pred_best,
    target_names=["low_vol(0)", "high_vol(1)"],
))
print("VALID CM [0=low,1=high]:")
print(confusion_matrix(y_valid, y_valid_pred_best, labels=[0, 1]))

print("\n===== TEST @ best_th =====")
print("Accuracy:", accuracy_score(y_test, y_test_pred_best))
print(classification_report(
    y_test,
    y_test_pred_best,
    target_names=["low_vol(0)", "high_vol(1)"],
))
print("TEST CM [0=low,1=high]:")
print(confusion_matrix(y_test, y_test_pred_best, labels=[0, 1]))


In [ ]:
# Cell 13 (NEW): ใช้ threshold ที่เลือกไปเทสบน Test set

from sklearn.metrics import classification_report, confusion_matrix

BEST_TH = 0.35  # 👈 ปรับค่าตรงนี้ให้ตรงกับที่คุณเลือกจาก Cell 12

y_valid_pred = (y_valid_proba >= BEST_TH).astype(int)
y_test_pred = (y_test_proba >= BEST_TH).astype(int)

print(f"✅ VALID @ threshold = {BEST_TH}")
print("Accuracy:", accuracy_score(y_valid, y_valid_pred))
print(classification_report(
    y_valid,
    y_valid_pred,
    target_names=["low_vol(0)", "high_vol(1)"],
))

print("VALID CM [0=low,1=high]:")
print(confusion_matrix(y_valid, y_valid_pred, labels=[0, 1]))

print("\n✅ TEST @ threshold = ", BEST_TH)
print("Accuracy:", accuracy_score(y_test, y_test_pred))
print(classification_report(
    y_test,
    y_test_pred,
    target_names=["low_vol(0)", "high_vol(1)"],
))

print("TEST CM [0=low,1=high]:")
print(confusion_matrix(y_test, y_test_pred, labels=[0, 1]))


In [ ]:
# Cell 14 (UPDATED): ดู feature สำคัญของ Regime Model

importances = rf_regime.feature_importances_
idx = np.argsort(importances)[::-1]

print("🔎 Feature importance (มาก → น้อย):\n")
for i in idx:
    print(f"{feature_cols[i]:30s} : {importances[i]:.4f}")
